In [2]:
# Import dependencies
import os
import json

In [3]:
%pip install --quiet "evadb[document,notebook]"
import evadb
cursor = evadb.connect().cursor()

Note: you may need to restart the kernel to use updated packages.


/Users/hershdhillon23/projects/evadb/evadb-venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Downloading: "http://ml.cs.tsinghua.edu.cn/~chenxi/pytorch-models/mnist-b07bb66b.pth" to /Users/hershdhillon23/.cache/torch/hub/checkpoints/mnist-b07bb66b.pth
100%|██████████| 1.03M/1.03M [00:01<00:00, 697kB/s] 
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /Users/hershdhillon23/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


In [56]:
# Set your OpenAI key as an environment variable
import os
#os.environ['OPENAI_API_KEY'] = ''
open_ai_key = os.environ.get("OPENAI_API_KEY", "")


In [77]:
# set up the extract columns UDF available at functions/extract_columns.py
cursor.query("""CREATE FUNCTION IF NOT EXISTS ExtractColumn
            IMPL  '../evadb/functions/extract_column.py';
                """).execute()

In [78]:
# # delete the table if it already exists
cursor.query("""DROP TABLE IF EXISTS InputUnstructured
                """).execute()

# create the table specifying the type of the prompt column
cursor.query("""CREATE TABLE IF NOT EXISTS InputUnstructured (
             input_rows TEXT)
                """).execute()

table = cursor.query("SELECT * FROM InputUnstructured;").df()
print(table)

Empty DataFrame
Columns: []
Index: []


In [79]:
input_rows_list = ["The touch screen on my tablet stopped working for no reason.",
#                   "Why does my computer take so long to start up? It's been like this for weeks.",
#                   "My phone battery dies too quickly. I just bought it!",
                    "My headphones won't connect to my phone anymore, even though they used to work just fine.",
                   "The software update completely messed up my computer. Now nothing works properly."]

for input_row in input_rows_list:
    cursor.query(f"""INSERT INTO InputUnstructured (input_rows) VALUES ("{input_row}")""").execute()


In [80]:
table = cursor.query("SELECT * FROM InputUnstructured;").df()
print(table)

   _row_id                                         input_rows
0        1  The touch screen on my tablet stopped working ...
1        2  My headphones won't connect to my phone anymor...
2        3  The software update completely messed up my co...


In [82]:
table = cursor.query(
        """SELECT ExtractColumn("Issue Component","The component that is causing the issue", "string less than 2 words", input_rows) FROM InputUnstructured;"""
    ).df()

for _, row in table.iterrows():
    print(row['response'])
#print(table.iloc[1]['response'])


            You are given a user query. Your task is to extract the following fields from the query and return the result in string format.
            IMPORTANT: RETURN ONLY THE EXTRACTED VALUE (one word or phrase). DO NOT RETURN THE FIELD NAME OR ANY OTHER INFORMATION.
        

            Extract the following fields from the unstructured text below:
            
            Format of the field is given in the format
            Field Name: Field Description: Field Type
            Issue Component: The component that is causing the issue: string less than 2 words

            The unstructured text is as follows:
        
The touch screen on my tablet stopped working for no reason.
My headphones won't connect to my phone anymore, even though they used to work just fine.
The software update completely messed up my computer. Now nothing works properly.
touch screen
headphones, connect, phone, work, fine
software update
